### Работа с пространственными связями в описании сцен

работаем с предобработанным датасетом

структура датасета

```
{
    "description": "Тяжелая гантель лежит на деревянной тяжелой скамье рядом с тяжелой металлической штангой."
    "relation": ["гантель", "на", "скамья"],
    "target": "на"
}

{
    "description": "Тяжелая гантель лежит на деревянной тяжелой скамье рядом с тяжелой металлической штангой."
    "relation": ["гантель", "на", "штанга"],
    "target": "нет связи"
}

```

задача модели - по заданному тексту ("description") и заданной паре (["гантель", "скамья"]) угадать пространственную связь ("на")


модель T5rus с промптом + LoRA - потому что мы будем генерировать связь по паре объектов

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!ls -la /content/drive/MyDrive/VKR/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 24
drwx------ 3 root root 4096 Apr 27 05:50 dataset
drwx------ 3 root root 4096 Apr 27 05:53 library
drwx------ 2 root root 4096 Apr 29 09:32 metrics_T5ru
drwx------ 2 root root 4096 Apr 29 09:32 T5ru_lora_outputs
drwx------ 2 root root 4096 Apr 30 17:28 T5ru_PsC_lora_outputs
drwx------ 2 root root 4096 May  2 19:49 T5ru_spacial_lora_outputs


In [2]:
!pip install -U spacy > /dev/null 2>&1
!python -m spacy download ru_core_news_sm > /dev/null 2>&1
!pip install wandb > /dev/null 2>&1
!pip install datasets > /dev/null 2>&1

In [3]:
import transformers
import datasets
import huggingface_hub
import torch
import wandb

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(torch.__version__)

4.51.3
3.5.1
0.30.2
2.6.0+cu124


In [14]:
import json
import torch
import os
import sys
import warnings
import random
import glob

import numpy as np

from pathlib import Path
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import TrainerCallback

from peft import LoraConfig, get_peft_model, TaskType, PeftConfig,PeftModel
from tqdm import tqdm


# отключаем их все чтобы картинку не портили
warnings.filterwarnings("ignore", category=FutureWarning)

DATA_DIR = Path("/content/drive/MyDrive/VKR/dataset/dataset_syntetic_v5_spacial_only").expanduser()
#DATA_DIR = Path("/content/drive/MyDrive/VKR/dataset/dataset_small").expanduser()
MODEL_NAME = "sberbank-ai/ruT5-base"

MAX_INPUT_LENGTH = 512 # максимальная длина входа
MAX_OUTPUT_LENGTH = 32 # максимальная длина выхода

lib_path = os.path.abspath(os.path.join(os.getcwd(), '/content/drive/MyDrive/VKR/'))
sys.path.append(lib_path)

from library.metrics_pseudo import evaluate_relation_predictions, evaluate_relation_predictions
from library.safe_compute_metrics import safe_compute_metrics

# перевод в псевдотекст и обратно


In [15]:
PROMPT_TEMPLATE = """
Определи пространственную связь между объектами '{obj1}' и '{obj2}'
в следующем описании сцены: {description}"""


In [16]:
def load_dataset(path=DATA_DIR):
    all_data = []
    for file in glob.glob(f"{path}/spatial_relations_batch_*.jsonl"):
        with open(file, 'r', encoding='utf-8') as f:
            for line in f:
                item = json.loads(line)
                obj1, obj2 = item["relation"]
                prompt = PROMPT_TEMPLATE.format(obj1=obj1, obj2=obj2, description=item["description"])
                all_data.append({
                    "input": prompt,
                    "target": item["target"],
                    "relation": item["relation"],
                    "description": item["description"]
                })
    return Dataset.from_list(all_data)


In [17]:
load_dataset()

Dataset({
    features: ['input', 'target', 'relation', 'description'],
    num_rows: 21671
})

In [20]:
lora_rank = 8
lora_alpha = 16
lora_target_modules=["q", "v"]   # в какие слои делаем инъекции
lora_dropout=0.1

start_learning_rate = 5e-4 # стартовый
lr_scheduler_type="cosine"
warmup_steps=1000 # прогрев (меньше чем эпоха в нашем случае)

per_device_train_batch_size = 16
num_train_epochs = 30

# параметры генерации
NUM_BEAMS = 6


In [21]:
run = wandb.init(
    entity="shiltsov-da",
    # Set the wandb project where this run will be logged.
    project="vkr-hse-object-detection",
    # Track hyperparameters and run metadata.
    group="T5LoRAspacial",
    tags=["spacial", "lora", MODEL_NAME],
    config={
        "architecture": "T5ru-LoRA-spacial",
        "notebook":"T5ru-LoRA-spacial-v3-Colab.ipynb",
        "base_model": MODEL_NAME,
        "lora_rank": lora_rank,
        "lora_alpha": lora_alpha,
        "lora_target_modules": lora_target_modules,
        "lora_dropout": lora_dropout,
        "start_learning_rate": start_learning_rate,
        "lr_scheduler_type": lr_scheduler_type,
        "lr_warmup_steps": warmup_steps,
        "per_device_train_batch_size": per_device_train_batch_size,
        "num_train_epochs": num_train_epochs
    },
)


In [22]:
def tokenize(example):
    inputs = tokenizer(example["input"], truncation=True, padding="max_length", max_length=MAX_INPUT_LENGTH)
    targets = tokenizer(example["target"], truncation=True, padding="max_length", max_length=MAX_OUTPUT_LENGTH)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

raw_dataset = load_dataset()
tokenized_dataset = raw_dataset.map(tokenize, batched=False)

train_test = tokenized_dataset.train_test_split(test_size=0.05)
train_ds = train_test['train']
eval_ds = train_test['test']


Map:   0%|          | 0/21671 [00:00<?, ? examples/s]

In [23]:
def compute_metrics(eval_preds):

    predictions, labels = eval_preds

    # Если predictions — logits, нужно брать argmax
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    # logits -> ids
    predictions = np.argmax(predictions, axis=-1)

    # Декодируем токены
    pred_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    print(pred_texts[:10],"\n" ,label_texts[:10],"\n")

    results = [
        {"relation": eval_ds[i]["relation"], "target": label_texts[i], "predicted_target": pred_texts[i]}
        for i in range(len(label_texts))
    ]

    #for i in range(min(3, len(results))):
    #    print(f"[{i}] relation: {results[i]['relation']}, target: {results[i]['target']}, pred: {results[i]['predicted_target']}")

    metrics = evaluate_relation_predictions(results)
    return {
        "F1binary": metrics["F1binary"],
        "F1strict": metrics["F1strict"]
    }


In [12]:
peft_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(base_model, peft_config)

In [24]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/VKR/T5ru_spacial_lora_outputs",
    logging_dir="/content/drive/MyDrive/VKR/logs_T5ru_spacial",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=start_learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_steps=warmup_steps,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=4,
    eval_accumulation_steps=3, # иначе все не влезет
    num_train_epochs=num_train_epochs,
    report_to="wandb",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# грузим уже обученную
#trainer.train(resume_from_checkpoint=True)

trainer.train()
model.save_pretrained("/content/drive/MyDrive/VKR/T5ru_spacial_lora_outputs")
run.finish()


No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,F1binary,F1strict
1,0.029300,0.014498,0.890100,0.860700
2,0.013900,0.006958,0.968100,0.935400
3,0.011400,0.005606,0.972000,0.944600
4,0.009600,0.006195,0.952900,0.934500
5,0.008300,0.006762,0.965200,0.943700
6,0.008200,0.006016,0.961400,0.941900
7,0.008200,0.004839,0.975900,0.950200
8,0.007500,0.005191,0.967200,0.950200
9,0.006800,0.005290,0.979800,0.954800
10,0.006100,0.005478,0.963300,0.946500


['нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'перед', 'нет связи'] 
 ['нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'перед', 'нет связи'] 

['нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'перед', 'нет связи'] 
 ['нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'перед', 'нет связи'] 

['нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'перед', 'нет связи'] 
 ['нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'перед', 'нет связи'] 

['нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'перед', 'нет связи'] 
 ['нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет связи', 'нет 

eval/F1binary,▁▇▇▆▇▆█▇█▇▇█▇█▇█▇█▇▇▇█▇███████
eval/F1strict,▁▆▇▆▇▇▇▇▇▇▇▇▇▇▇████▇▇█▇███████
eval/loss,█▃▂▂▃▂▁▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▇▆▄▃▅█▅▆▅▇▆▅▇▇▆▇▅█▃▄▄▅▂▂▂▃▂▃▂▁
eval/samples_per_second,▂▃▅▆▄▁▄▃▄▂▃▄▂▂▃▂▄▁▆▅▅▄▇▇▇▆▇▆▇█
eval/steps_per_second,▂▃▅▆▄▁▄▃▄▂▃▄▂▂▃▂▄▁▆▅▅▄▇▇▇▆▇▆▇█
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇████
train/grad_norm,▃▆▃▅▄▃▄▃▃▅▃▂▂▁▂▂▁█▃▃▁▁▂█▅▁▃▁▁▁▁▁▃▃▁▄▁▃▃▁
train/learning_rate,▄█████████▇▇▇▇▇▇▆▆▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▁▁▁▁▁
train/loss,█▆▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [14]:
# на всякий случай
run.finish()

### Инференс

In [17]:
def object_relation_inference(description: str, obj1: str, obj2: str) -> str:
    model.eval()
    prompt = PROMPT_TEMPLATE.format(obj1=obj1, obj2=obj2, description=description)
    print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length = MAX_INPUT_LENGTH).to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=10)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [19]:
description = "кот сидит рядом со стулом"
obj1 = "стол"
obj2 = "кот"

object_relation_inference(description, obj1, obj2)


Определи пространственную связь между объектами 'стол' и 'кот' 
в следующем описании сцены: кот сидит рядом со стулом


'нет связи'